In [4]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def getOlx():
    olx_list = []
    for i in tqdm(range(1,101), desc='OLX'):
        r = requests.get(f'https://www.olx.com.br/imoveis/estado-sp?o={i}', headers=headers)
        content = r.content
        soup = BeautifulSoup(r.content, 'html.parser') 
        li_tags = soup.find_all('li')
        # Iterando sobre as tags <li> encontradas
        for li_tag in li_tags:
            # Verificando se a tag <li> contém um link
            if li_tag.find('a'):
                # Obtendo o valor do atributo "href" do link
                link = li_tag.find('a')['href']        
                if link.startswith('https://sp.olx.com.br'):
                    olx_list.append(link)
    return olx_list

def getLoft():
    temp = []
    loft_list = []
    for i in tqdm(range(1, 133),desc='Loft'):
            response = requests.get(f'https://loft.com.br/venda/imoveis/sp/sao-paulo?utm_source=google&utm_medium=cpc&utm_campaign=ins_01_br_001_sp_0001_sao-paulo_all_aw_search_conversion_broad_&utm_content=all_loft-pura&utm_id=1756086660&utm_placement=&utm_ad_id=576438774862&gclid=Cj0KCQjwjryjBhD0ARIsAMLvnF9KvelK1iKF7xoTJ3HS0KSWMhoDO0UlANc1gTfQEMGg5Bud0HiPZg4aAscOEALw_wcB&pagina={i}')
            html_content = response.text

            soup = BeautifulSoup(html_content, 'html.parser')

            divs = soup.find_all('div', class_='MuiGrid-root')

            for div in divs:
                tag_a = div.find('a')
                if tag_a is not None:
                    link = tag_a['href']
                    if link.startswith('/imovel/'):
                        temp.append(link)
    loft_list = ['http://loft.com.br' + valor for valor in temp]
    return loft_list

## MAIN CODE
olx_list = getOlx()
loft_list = getLoft()


Loft: 100%|██████████| 132/132 [00:25<00:00,  5.25it/s]


In [33]:
r = requests.get(f'https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/apartamento-253m--4-quartos-pinheiros-1191032220', headers=headers)
content = r.content
soup = BeautifulSoup(r.content, 'html.parser') 
div = soup.find_all('div', class_='sc-bwzfXH ad__h3us20-0 ikHgMx')



In [35]:
categorias = []
tipos = []
dados = []

# Encontrar todas as tags <dd>, <dt> e <a> dentro da div
dds = div.find_all('dd')
dts = div.find_all('dt')
as_ = div.find_all('a')

# Iterar sobre as tags e extrair os dados
for dd, dt, a in zip(dds, dts, as_):
    categoria = dd.text.strip()
    
    if categoria == 'categoria' or categoria == 'tipo':
        tipo = a.text.strip()
        dados.append(tipo)
    else:
        tipo = dt.text.strip()
        dados.append(tipo)
    
    categorias.append(categoria)
    tipos.append(tipo)

# Imprimir as listas de dados
print("Categorias:", categorias)
print("Tipos:", tipos)
print("Dados:", dados)

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
sc-bwzfXH ad__h3us20-0 ikHgMx

sc-bwzfXH ad__h3us20-0 ikHgMx
sc-bwzfXH ad__h3us20-0 ikHgMx

sc-bwzfXH ad__h3us20-0 ikHgMx

In [11]:
test2

[<dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">05428000</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">São Paulo</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">Pinheiros</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">Rua Ferreira de Araújo - até 385/386</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">R$ 3.302</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">R$ 1.161</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">254m²</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">4</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">3</dd>,
 <dd class="ad__sc-1f2ug0x-1 cpGpXB sc-ifAKCX kaNiaQ" color="dark" font-weight="400">Pa

In [15]:
test3 = test2[0].text
test3

'05428000'

In [16]:
olx_list

['https://sp.olx.com.br/baixada-santista-e-litoral-sul/imoveis/aluga-se-apto-no-marape-1180368205',
 'https://sp.olx.com.br/regiao-de-presidente-prudente/comercio-e-industria/passo-ponto-comercial-1170774758',
 'https://sp.olx.com.br/regiao-de-presidente-prudente/comercio-e-industria/mercado-acougue-e-padaria-em-aracatuba-1189727771',
 'https://sp.olx.com.br/vale-do-paraiba-e-litoral-norte/imoveis/apartamento-3-suites-venda-em-belvedere-jardim-das-colinas-sao-jose-dos-campos-sp-1187146960',
 'https://sp.olx.com.br/baixada-santista-e-litoral-sul/imoveis/apartamento-com-2-dormitorios-86-m-venda-por-r-580-000-00-ou-aluguel-por-r-3-200-00-1191032478',
 'https://sp.olx.com.br/regiao-de-sorocaba/imoveis/casa-de-condominio-para-venda-tem-320-metros-quadrados-com-4-quartos-1191032465',
 'https://sp.olx.com.br/baixada-santista-e-litoral-sul/imoveis/apartamento-com-3-quartos-a-venda-89m-enseada-1191032436',
 'https://sp.olx.com.br/baixada-santista-e-litoral-sul/imoveis/apartamento-com-1-dormitor